In [1]:
import pandas as pd
import numpy as np

In [2]:
import dask.dataframe as dd

In [6]:
ddf = dd.read_csv("train_data_feeds.csv", dtype={'u_newsCatDislike': 'object'})

In [4]:
dda = dd.read_csv("train_data_ads.csv")

In [8]:
dda.head()

,log_id,label,user_id,age,gender,residence,city,city_rank,series_dev,series_group,...,ad_click_list_v001,ad_click_list_v002,ad_click_list_v003,ad_close_list_v001,ad_close_list_v002,ad_close_list_v003,pt_d,u_newsCatInterestsST,u_refreshTimes,u_feedLifeCycle
0,373250,0,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
1,373253,1,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
2,373252,1,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
3,373251,0,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
4,373255,0,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030328,39^220^16,0,15


In [9]:
ddf.head()

,u_userId,u_phonePrice,u_browserLifeCycle,u_browserMode,u_feedLifeCycle,u_refreshTimes,u_newsCatInterests,u_newsCatDislike,u_newsCatInterestsST,u_click_ca2_news,...,e_ch,e_m,e_po,e_pl,e_rn,e_section,e_et,label,cillabel,pro
0,135880,16,17,10,17,0,195^168^109^98^108,0,195^44^168^112^21,195^168^44^112^21,...,19,1217,1,561,2,0,202206081521,-1,-1,0
1,135880,16,17,10,17,0,195^168^109^98^108,0,195^44^168^112^21,195^168^44^112^21,...,19,1217,9,561,1,0,202206081521,-1,-1,0
2,135880,16,17,10,17,0,195^168^109^98^108,0,195^44^168^112^21,195^168^44^112^21,...,19,1217,18,561,1,0,202206081521,-1,-1,0
3,135880,16,17,10,17,0,195^168^109^98^108,0,195^44^168^112^21,195^168^44^112^21,...,19,1217,7,561,1,1,202206081521,-1,-1,0
4,135880,16,17,10,17,0,195^168^109^98^108,0,195^44^168^112^21,195^168^44^112^21,...,19,1217,7,561,2,0,202206081522,-1,-1,0


In [14]:
ddf.compute()['u_userId'].unique()

array([135880, 116738, 109694, ..., 216655, 127804, 137663], dtype=int64)

In [15]:
dda.compute()['user_id'].unique()

array([100005, 100006, 100009, ..., 286534, 286715, 286999], dtype=int64)

In [16]:
ddf.compute().shape

(3227732, 28)

In [17]:
dda.compute().shape

(7675517, 35)

In [18]:
ddf['user_id'] = ddf['u_userId']

In [19]:
ddf = ddf.drop('u_userId', axis=1)

In [20]:
merged_df = ddf.merge(dda, on='user_id', how='outer')

In [21]:
merged_df.head()

,u_phonePrice,u_browserLifeCycle,u_browserMode,u_feedLifeCycle_x,u_refreshTimes_x,u_newsCatInterests,u_newsCatDislike,u_newsCatInterestsST_x,u_click_ca2_news,i_docId,...,ad_click_list_v001,ad_click_list_v002,ad_click_list_v003,ad_close_list_v001,ad_close_list_v002,ad_close_list_v003,pt_d,u_newsCatInterestsST_y,u_refreshTimes_y,u_feedLifeCycle_y
0,14,17,14,15,0,168^109^130^114^112,0,219,130^83^114^65^144,f3d756b2adb5a197da20730bebdc7fb566cab202,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14,17,14,15,0,168^109^130^114^112,0,219,130^83^114^65^144,45d29e335c0ed9633c2e84e993131cfaf3a14632,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,17,14,17,6,44^168^44^207^213,0,219^152^57^102^98,152^102^57^78^98,75459822c533c79b12094372769defe694d66b84,...,14543^21493^21965^26531^30157,1097^1036^1410^2039^2066,312^190^114^162,24107,1218,173,2.022061e+11,20^44^16^98^25,6.0,17.0
3,10,17,14,17,6,44^168^44^207^213,0,219^152^57^102^98,152^102^57^78^98,75459822c533c79b12094372769defe694d66b84,...,14543^21493^21965^26531^30157,1097^1036^1410^2039^2066,312^190^114^162,24107,1218,173,2.022061e+11,20^44^16^98^25,6.0,17.0
4,10,17,14,17,6,44^168^44^207^213,0,219^152^57^102^98,152^102^57^78^98,75459822c533c79b12094372769defe694d66b84,...,14543^21493^21965^26531^30157,1097^1036^1410^2039^2066,312^190^114^162,24107,1218,173,2.022061e+11,20^44^16^98^25,6.0,17.0


In [22]:
merged_df.compute().shape

MemoryError: 